In [1]:
from google.colab import drive
import os
import cv2
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your dataset folder in Google Drive
dataset_path = '/content/drive/MyDrive/dataset/painting'

# Function to load images in batches
def load_images_in_batches(folder_path, batch_size=100):
    """Loads images from a folder in batches.

    Args:
        folder_path (str): The path to the folder containing images.
        batch_size (int, optional): The size of each batch. Defaults to 100.

    Yields:
        numpy.ndarray: A batch of images as a NumPy array.
    """
    filenames = os.listdir(folder_path)
    for i in range(0, len(filenames), batch_size):
        batch_filenames = filenames[i:i + batch_size]
        images = []
        for filename in batch_filenames:
            file_path = os.path.join(folder_path, filename)

            try:
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Warning: Unable to read image {file_path}. Skipping.")
                    continue
                img = cv2.resize(img, (256, 256))
                img = img.astype('float32') / 255.0
                images.append(img)
            except cv2.error as e:
                print(f"Error processing image {file_path}: {e}")

        if images:  # Check if images list is not empty
            yield np.array(images)

# Example usage: Iterate through batches of images
for image_batch in load_images_in_batches(dataset_path, batch_size=100):
    print(f"Processing batch of size: {image_batch.shape}")

Mounted at /content/drive
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (99, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (100, 256, 256, 3)
Processing batch of size: (42, 256, 256, 3)


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Contextual Attention Mechanism
class ContextualAttention(layers.Layer):
    def __init__(self, filters, kernel_size):
        super(ContextualAttention, self).__init__()
        self.conv_x = layers.Conv2D(filters, kernel_size, padding='same')
        self.conv_mask = layers.Conv2D(filters, kernel_size, padding='same')

    def call(self, x, mask):
        # Resize the mask to match the dimensions of the image features
        mask = tf.image.resize(mask, tf.shape(x)[1:3])

        # Apply convolution on the image and the mask
        x = self.conv_x(x)
        mask = self.conv_mask(mask)

        # Element-wise multiplication and softmax to calculate attention
        attention = tf.nn.softmax(x * mask, axis=-1)
        return attention * x

# Encoder-Decoder Architecture with Contextual Attention
def build_model():
    # Inputs: image and mask
    input_image = layers.Input(shape=(256, 256, 3))
    input_mask = layers.Input(shape=(256, 256, 1))

    # Encoder: Downsample the image
    x = layers.Conv2D(64, (4, 4), activation='relu', padding='same')(input_image)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (4, 4), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Apply Contextual Attention
    att = ContextualAttention(128, (4, 4))(x, input_mask)

    # Decoder: Upsample the attended feature maps
    x = layers.Conv2DTranspose(128, (4, 4), strides=2, activation='relu', padding='same')(att)
    x = layers.Conv2DTranspose(64, (4, 4), strides=2, activation='relu', padding='same')(x)

    # Output: Reconstructed image
    output_image = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Define the model
    model = models.Model(inputs=[input_image, input_mask], outputs=output_image)
    return model

# Compile the Model
model = build_model()
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])

# Print the Model Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │          3,136 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 128, 128)  │        131,200 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 256, 256, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ contextual_attention      │ (None, 64, 64, 128)    │        264,448 │ max_pooling2d_1[0][0], │
│ (ContextualAttention)     │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 128, 128, 128)  │        262,272 │ contextual_attention[… │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_1        │ (None, 256, 256, 64)   │        131,136 │ conv2d_transpose[0][0] │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 256, 256, 3)    │          1,731 │ conv2d_transpose_1[0]… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 793,923 (3.03 MB)

 Trainable params: 793,923 (3.03 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import tensorflow as tf
from tensorflow.keras import backend as K

def ssim_loss(y_true, y_pred):
    return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)

def combined_loss(y_true, y_pred):
    l1 = K.mean(K.abs(y_true - y_pred))
    ssim = ssim_loss(y_true, y_pred)
    return l1 + ssim

In [4]:
import os
import cv2
import shutil
import numpy as np

# --- Paths ---
dataset_path = '/content/drive/MyDrive/dataset/painting'
masks_path = os.path.join(dataset_path, 'masks')

# --- Create directories ---
for path in [masks_path]:
    os.makedirs(path, exist_ok=True)

# --- Function to create masks ---
def create_masks(images_path, masks_path):
    """Creates binary masks for images and saves them."""
    for filename in os.listdir(images_path):
        image_path = os.path.join(images_path, filename)
        try:
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Unable to read image {image_path}. Skipping.")
                continue

            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

            mask_filename = os.path.splitext(filename)[0] + '.jpg'
            mask_save_path = os.path.join(masks_path, mask_filename)
            cv2.imwrite(mask_save_path, mask)

            print(f"Created and saved mask for: {filename}")

        except cv2.error as e:
            print(f"Error processing image {image_path}: {e}")

#  Create Masks ---
create_masks(dataset_path, masks_path)

Created and saved mask for: 1218.jpg
Created and saved mask for: 1226.jpg
Created and saved mask for: 1211.jpg
Created and saved mask for: 1198.jpg
Created and saved mask for: 1206.jpg
Created and saved mask for: 1201.jpg
Created and saved mask for: 1207.jpg
Created and saved mask for: 1204.jpg
Created and saved mask for: 1221.jpg
Created and saved mask for: 1196.jpg
Created and saved mask for: 1220.jpg
Created and saved mask for: 1224.jpg
Created and saved mask for: 1203.jpg
Created and saved mask for: 1197.jpg
Created and saved mask for: 1219.jpg
Created and saved mask for: 1214.jpg
Created and saved mask for: 1216.jpg
Created and saved mask for: 1247.jpg
Created and saved mask for: 1252.jpg
Created and saved mask for: 1240.jpg
Created and saved mask for: 1244.jpg
Created and saved mask for: 1245.jpg
Created and saved mask for: 1238.jpg
Created and saved mask for: 1230.jpg
Created and saved mask for: 1231.jpg
Created and saved mask for: 1229.jpg
Created and saved mask for: 1254.jpg
C

In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# --- Paths ---
dataset_path = '/content/drive/MyDrive/dataset/painting'
masks_path = os.path.join(dataset_path, 'masks')

target_size = (256, 256)

def load_data(images_path, masks_path, img_size=(256, 256)):
    """Loads and preprocesses images and masks from specified paths.

    Args:
        images_path (str): Path to the folder containing the images.
        masks_path (str): Path to the folder containing the masks.
        img_size (tuple, optional): Target size for resizing images. Defaults to (256, 256).

    Returns:
        tuple: A tuple containing NumPy arrays of images and masks.
    """
    images = []
    masks = []

    for filename in os.listdir(images_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            mask_path = os.path.join(masks_path, filename)

            if not os.path.exists(mask_path):
                print(f"Error: Mask not found for {filename} at {mask_path}")
                continue

            try:
                # --- Image ---
                print(f"Processing: {filename}")
                img_path = os.path.join(images_path, filename)
                image = load_img(img_path, target_size=img_size)
                image = img_to_array(image) / 255.0
                images.append(image)

                # --- Mask ---
                print(f"Mask path: {mask_path}")
                mask = load_img(mask_path, target_size=img_size, color_mode='grayscale')
                mask = img_to_array(mask) / 255.0
                masks.append(mask)

            except FileNotFoundError:
                print(f"Warning: Image not found for {filename}. Skipping.")

    return np.array(images), np.array(masks)

# --- Load the data ---
images, masks, = load_data(dataset_path, masks_path, img_size=target_size)

Processing: 1218.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1218.jpg
Processing: 1226.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1226.jpg
Processing: 1211.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1211.jpg
Processing: 1198.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1198.jpg
Processing: 1206.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1206.jpg
Processing: 1201.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1201.jpg
Processing: 1207.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1207.jpg
Processing: 1204.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1204.jpg
Processing: 1221.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1221.jpg
Processing: 1196.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1196.jpg
Processing: 1220.jpg
Mask path: /content/drive/MyDrive/dataset/painting/masks/1220.jpg
Processing: 1224.jpg
Mask path: /content/dr

In [6]:
import os
import cv2
import numpy as np

# --- Paths ---
dataset_path = '/content/drive/MyDrive/dataset/painting'
masks_path = os.path.join(dataset_path, 'masks')

target_size = (256, 256)

def load_data(images_path, masks_path, img_size=(256, 256)):

    images = []
    masks = []

    for filename in os.listdir(images_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(images_path, filename)
            mask_path = os.path.join(masks_path, filename)  # Assuming masks have the same filename

            try:
                image = cv2.imread(image_path)
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

                if image is None:
                    print(f"Warning: Image not found for {filename} at {image_path}. Skipping.")
                    continue
                if mask is None:
                    print(f"Warning: Mask not found for {filename} at {mask_path}. Skipping.")
                    continue

                image = cv2.resize(image, img_size)
                mask = cv2.resize(mask, img_size)

                # Normalize and convert to float32
                image = image.astype('float32') / 255.0
                mask = mask.astype('float32') / 255.0

                images.append(image)
                masks.append(mask)

            except cv2.error as e:
                print(f"Error processing {filename}: {e}")

    return np.array(images), np.array(masks)

# --- Load the data ---
images, masks, = load_data(dataset_path, masks_path, img_size=target_size)

In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# --- Paths ---
dataset_path = '/content/drive/MyDrive/dataset/painting'
masks_path = os.path.join(dataset_path, 'masks')

target_size = (256, 256)

# --- Function to load images and masks ---
def load_data(images_path, masks_path, img_size=(256, 256)):

    images = []
    masks = []

    for filename in os.listdir(images_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(images_path, filename)
            mask_path = os.path.join(masks_path, filename)

            try:
                image = cv2.imread(image_path)
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

                if image is None:
                    print(f"Warning: Image not found for {filename} at {image_path}. Skipping.")
                    continue
                if mask is None:
                    print(f"Warning: Mask not found for {filename} at {mask_path}. Skipping.")
                    continue

                image = cv2.resize(image, img_size)
                mask = cv2.resize(mask, img_size)

                # Normalize and convert to float32
                image = image.astype('float32') / 255.0
                mask = mask.astype('float32') / 255.0

                images.append(image)
                masks.append(mask)

            except cv2.error as e:
                print(f"Error processing {filename}: {e}")

    return np.array(images), np.array(masks)


# --- Load the data ---
images, masks = load_data(dataset_path, masks_path, img_size=target_size)

# --- Split the data ---
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

# --- Build and compile the model (assuming 'build_model' and 'combined_loss' are defined) ---
model = build_model()
model.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])

# --- Train the model ---
batch_size = 8
epochs = 3
history = model.fit(
    [X_train, y_train],
    X_train,  # Use the input images as the target
    validation_data=([X_test, y_test], X_test),
    batch_size=batch_size,
    epochs=epochs
)

Training set shape: (1632, 256, 256, 3)
Testing set shape: (409, 256, 256, 3)
Epoch 1/3
204/204 ━━━━━━━━━━━━━━━━━━━━ 3812s 19s/step - accuracy: 0.5750 - loss: 0.5050 - val_accuracy: 0.8333 - val_loss: 0.1723
Epoch 2/3
204/204 ━━━━━━━━━━━━━━━━━━━━ 3804s 18s/step - accuracy: 0.8299 - loss: 0.1529 - val_accuracy: 0.8518 - val_loss: 0.1236
Epoch 3/3
204/204 ━━━━━━━━━━━━━━━━━━━━ 3808s 18s/step - accuracy: 0.8499 - loss: 0.1175 - val_accuracy: 0.8535 - val_loss: 0.1089


In [10]:
# --- Calculate PSNR ---
y_pred = model.predict([X_test, y_test])
psnr = tf.image.psnr(X_test, y_pred, max_val=1.0)
print(f'PSNR: {tf.reduce_mean(psnr).numpy():.4f}')

# --- Calculate SSIM ---
ssim = tf.image.ssim(X_test, y_pred, max_val=1.0)
print(f'SSIM: {tf.reduce_mean(ssim).numpy():.4f}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 148s 11s/step
PSNR: 30.7573
SSIM: 0.9118


In [9]:
# --- Evaluate the model ---
loss, accuracy = model.evaluate([X_test, y_test], X_test, verbose=0)
print(f'Test loss (MAE): {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# --- Calculate combined loss ---
y_pred = model.predict([X_test, y_test])
combined_loss_value = combined_loss(X_test, y_pred).numpy()
mean_combined_loss = np.mean(combined_loss_value)
print(f'Combined loss: {mean_combined_loss:.4f}')



Test loss (MAE): 0.1089
Test accuracy: 0.8535
13/13 ━━━━━━━━━━━━━━━━━━━━ 150s 12s/step
Combined loss: 0.1089
